In [1]:
import PySimpleGUI as sg
from pygame import mixer, time
import pygame
from pydub import AudioSegment
from pydub.playback import play
import numpy as np
import io
import os
import sounddevice as sd
import librosa
import threading

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
sg.theme('GreenTan')

'GreenTan'

In [3]:
def win():
    layout=[[sg.Text("Choose a status"),sg.Button('Wav Pitch'),
            sg.Button('MIC Pitch')]]
    return sg.Window('GUI',layout,relative_location=(0,-100),finalize=True)

In [4]:
def win1():
    layout=[[sg.Text("Choose a file: "), sg.Input(), sg.FileBrowse(key="-SOUND_PATH-")],
    [sg.Text(size=(12,1), key='-STATUS-')],
    [sg.Button('Play', pad=(10, 0), key='-PLAY-'),sg.Button('Stop', pad=(10, 0), key='-STOP-'),
    sg.Text('Pitch: '),sg.Slider(range=(-12,12), orientation='h', size=(50, 20), enable_events=True, key = '-PITCH-', default_value= 1.0,resolution=1.0)
    ]]
    return sg.Window('WAV FILE',layout,relative_location=(0,-100),finalize=True)

In [5]:
def win2():
    layout=[[sg.Text("MIC PITCH SHIFTER"),
    sg.Text('Pitch: '),sg.Slider(range=(-12,12), orientation='h', size=(50, 20), enable_events=True, key = '-PITCH-', default_value= 1.0,resolution=1.0)
    ]]
    return sg.Window('MIC PITCH',layout,relative_location=(0,-100),finalize=True)

In [6]:
def change_pitch(audio, semitones):
    return audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate * (2 ** (semitones / 12.0)))
    })

In [7]:
def pitch_shift_callback(indata, outdata, frames, time, status):
    if status:
        print('Status:', status)
    audio_data = np.frombuffer(indata, dtype=np.float32)
    shifted_audio = librosa.effects.pitch_shift(audio_data, sr=RATE, n_steps=PITCH_SHIFT)
    if len(shifted_audio) < len(indata):
        shifted_audio = np.pad(shifted_audio, (0, len(indata) - len(shifted_audio)))
    else:
        shifted_audio = shifted_audio[:len(indata)]
    outdata[:] = shifted_audio[:, np.newaxis]

In [8]:
mixer.init()
pygame.init()
is_playing = False

In [9]:
def pitch():
    window_pitch = win1()
    temp_file_path = "" 
    try:
        while True:
            event, values = window_pitch.read()
            if event == sg.WIN_CLOSED or event == "Exit":
                mixer.quit()
                temp_file_path = values['-SOUND_PATH-'] + 'temp.wav'
                break
            sound_path = values["-SOUND_PATH-"]
            if not sound_path:
                sg.Popup("No song specified.")
                continue
            pitch_value = values['-PITCH-']
            audio = AudioSegment.from_file(sound_path)
            modified_audio = change_pitch(audio, pitch_value)
            if event == '-PLAY-':
                window_pitch['-STATUS-'].update('Playing')
                is_playing = True
                mixer.music.stop()
                pygame.mixer.quit()
                temp_file_path = sound_path + 'temp.wav'
                modified_audio.export(temp_file_path, format="wav")
                mixer.init()
                mixer.music.load(temp_file_path)
                mixer.music.play()
            elif event == '-STOP-':
                mixer.music.stop()
                is_playing = False
    finally:
        window_pitch.close()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [10]:
def mic():
    window_mic = win2()
    CHUNK = 1024  # Audio chunk size
    FORMAT = np.float32
    CHANNELS = 1
    RATE = 44100  # Sample rate
    PITCH_SHIFT = 2  # Shift pitch by default
    def mic_callback(indata, outdata, frames, time, status):
        if status:
            print('Status:', status)
        audio_data = np.frombuffer(indata, dtype=np.float32)
        shifted_audio = librosa.effects.pitch_shift(audio_data, sr=RATE, n_steps=PITCH_SHIFT)
        # If the shifted audio length is less than the chunk size, pad it
        if len(shifted_audio) < len(indata):
            shifted_audio = np.pad(shifted_audio, (0, len(indata) - len(shifted_audio)))
        else:
            shifted_audio = shifted_audio[:len(indata)]

        # Copy shifted audio data to output buffer
        outdata[:] = shifted_audio[:, np.newaxis]

    def stream_thread():
        with sd.Stream(channels=CHANNELS, callback=mic_callback, samplerate=RATE, blocksize=CHUNK):
            print('Pitch shifting started. Press Ctrl+C to stop.')
            input("Press Enter to stop...")

    # Start the stream in a separate thread
    thread = threading.Thread(target=stream_thread)
    thread.start()

    while True:
        event, values = window_mic.read()

        if event == sg.WIN_CLOSED or event == "Exit":
            break

        if event == "-PITCH-":
            set_pitch_shift(values["-PITCH-"])

    # Wait for the stream thread to finish before closing the window
    thread.join()
    window_mic.close()


In [11]:
window_main=win()
while True:
        event, values = window_main.read()
        if event in (None, 'Quit'):
            break
        if event =='Wav Pitch':
            try:
                pitch()
            except:
                sg.popup('Instance Ended, Continuing to wait')
        elif event == 'MIC Pitch':
            try:
                mic()
            except:
                sg.popup('Instance Ended, Continuing to wait')
window.close()

Pitch shifting started. Press Ctrl+C to stop.


D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=

Status: input overflow, output underflow
Status: input overflow, output underflow


D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=

Press Enter to stop...


...

Pitch shifting started. Press Ctrl+C to stop.


D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=

NameError: name 'window' is not defined

Status: input overflow, output underflow
Status: input overflow, output underflow


D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
D:\anaconda\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=